# Fundamental Data Structures and Algorithms

> Materia: Algoritmos y Complejidad <br>
> Profesor: Juan Carlos Cueva Tello <br>
> Alumno: Jose Luis Rojas Aranda <br>
> Ing. Sistemas Inteligentes, UASLP <br>


In [1]:
import math
import numpy as np

## Topic: 6 Heapsort, Heaps

Like merge sort, but unlike insertion sort, heapsort’s running time is $O(nlgn)$. Like insertion sort, but unlike merge sort, heapsort sorts in place: only a constant number of array elements are stored outside the input array at any time. Thus, heapsort combines the better attributes of the two sorting algorithms we have already discussed.

Heapsort also introduces another algorithm design technique: using a data structure, in this case one we call a “heap,” to manage information. Not only is the heap data structure useful for heapsort, but it also makes an efficient priority queue.

### Heaps
The (binary) heap data structure is an array object that we can view as a nearly complete binary tree, as shown in Figure 6.1. 

<img src="imgs/CFig6_1.png" alt="drawing" width="600"/>

Each node of the tree corresponds to an element of the array. The tree is completely filled on all levels except possibly the lowest, which is filled from the left up to a point. An array `A` that represents a heap is an object with two attributes: `A.length`, which (as usual) gives the number of elements in the array, and `A.heapsize`, which represents how many elements in the heap are stored within array `A`. That is, although `A[1..A.lenght]` may contain numbers, only the elements in `A[1..A.heap-size]`, where `0 <= A.heap-size <= A.length`, are valid elements of the heap. The root of the tree is `A[1]` , and given the index `i` of a node, we can easily compute the indices of its parent, left child, and right child:

$Parent(i)$ **return** [i/2]

$Left(i)$ **return** 2i

$Right(i)$ **return** 2i + 1

There are two kinds of binary heaps: max-heaps and min-heaps. In both kinds, the values in the nodes satisfy a **heap property**, the specifics of which depend on the kind of heap. In a **max-heap**, the **max-heap property** is that for every node i other than the root,

$$A[Parent(i)] >= A[i]$$

that is, the value of a node is at most the value of its parent. Thus, the largest element in a max-heap is stored at the root, and the subtree rooted at a node contains values no larger than that contained at the node itself. A **min-heap** is organized in the opposite way; the **min-heap property** is that for every node i other than the root,

$$A[Parent(i)] <= A[i]$$

The smallest element in a min-heap is at the root.

In [2]:
class Heap:
    def __init__(self, arreglo):
        self.A = [None]
        for i in range(len(arreglo)):
            self.A.append(arreglo[i])
        self.heap_size = len(self.A) - 1
        self.length = self.heap_size

def Parent(i):
    return i/2

def Left(i):
    return (2*i)

def Right(i):
    return (2*i) + 1

### Maintaining the heap property
In order to maintain the max-heap property, we call the procedure `Max-Heapify`. Its inputs are an array A and an index `i` into the array. When it is called, `Max-Heapify` assumes that the binary trees rooted at $Left(i)$ and $Right(i)$ are max-heaps, but that `A[i]`  might be smaller than its children, thus violating the max-heap property. `Max-Heapify` lets the value at `A[i]`  “float down” in the max-heap so that the subtree rooted at index `i` obeys the max-heap property.

In [3]:
def Max_Heapify(h, i):
    l = Left(i)
    r = Right(i)
    if l <= h.heap_size and h.A[l] > h.A[i]:
        largest = l
    else:
        largest = i
    if r <= h.heap_size and h.A[r] > h.A[largest]:
        largest = r
    if largest != i:
        temp = h.A[i]
        h.A[i] = h.A[largest]
        h.A[largest] = temp
        Max_Heapify(h, largest)

Figure 6.2 illustrates the action of `Max-Heapify`.

<img src="imgs/CFig6_2.png" alt="drawing" width="500"/>

In [4]:
# Ejecutando ejemplo de la figura 6.2
heap = Heap([16, 4, 10, 14, 7, 9, 3, 2, 8, 1])
print("Original: {}".format(heap.A))
Max_Heapify(heap, 2)
print("Nodo 2: {}".format(heap.A))
Max_Heapify(heap, 4)
print("Nodo 4: {}".format(heap.A))

Original: [None, 16, 4, 10, 14, 7, 9, 3, 2, 8, 1]
Nodo 2: [None, 16, 14, 10, 8, 7, 9, 3, 2, 4, 1]
Nodo 4: [None, 16, 14, 10, 8, 7, 9, 3, 2, 4, 1]


The running time of `Max-Heapify` on a subtree of size $n$ rooted at a given node $i$ is the $\Theta(1)$ time to fix up the relationships among the elements $A[i]$, $A[Left(i)]$, and $A[Right(i)]$, plus the time to run`Max-Heapify` on a subtree rooted at one of the children of node $i$ (assuming that the recursive call occurs). The children’s subtrees each have size at most $2n/3$—the worst case occurs when the bottom level of the tree is exactly half full—and therefore we can describe the running time of `Max-Heapify` by the recurrence:

$$T(n) <= T(2n/3) + \Theta(1)$$

The solution to this recurrence, by case 2 of the master theorem, is $T(n)=O(lg n)$. Alternatively, we can characterize the running time of `Max-Heapify` on a node of height $h$ as $O(h)$.

### Building a heap
We can use the procedure `Max-Heapify` in a bottom-up manner to convert an array $A[1..n]$, where `n = A.length`, into a max-heap.

In [5]:
def Build_Max_Heap(h):
    for i in range(int(h.heap_size/2), 0, -1):
        Max_Heapify(h, i)

Figure 6.3 shows an example of the action of BUILD-MAX-HEAP.

<img src="imgs/CFig6_3.png" alt="drawing" width="400"/>

In [6]:
heap = Heap([4, 1, 3, 2, 16, 9, 10, 14, 8, 7])
Build_Max_Heap(heap)
print(heap.A)

[None, 16, 14, 10, 8, 7, 9, 3, 2, 4, 1]


### The heapsort algorithm
The heapsort algorithm starts by using `Build-Max-Heap` to build a max-heap on the input array $A[1..n]$ , where $n = A.length$. Since the maximum element of the array is stored at the root $A[1]$ , we can put it into its correct final position by exchanging it with $A[n]$ . If we now discard node $n$ from the heap—and we can do so by simply decrementing `A.heap-size`—we observe that the children of the root remain max-heaps, but the new root element might violate the max-heap property. All we need to do to restore the max-heap property, however, is call `Max-Heapify(A,1)`, which leaves a max-heap in $A[1..n-1]$. The heapsort algorithm then repeats this process for the max-heap of size $n-1$ down to a heap of size 2.

In [7]:
def Heapsort(h):
    Build_Max_Heap(h)
    for i in range(h.length, 1, -1):
        temp = h.A[i]
        h.A[i] = h.A[1]
        h.A[1] = temp
        h.heap_size = h.heap_size - 1
        Max_Heapify(h, 1)

The next figure shows an example of the operation of HEAPSORT

<img src="imgs/CFig6_4.png" alt="drawing" width="600"/>

In [8]:
Heapsort(heap)
print(heap.A)

[None, 1, 2, 3, 4, 7, 8, 9, 10, 14, 16]


The `Heapsort` procedure takes time $O(n lgn)$, since the call to `Build-Max-Heap` takes time $O(n)$ and each of the $n-1$ calls to `Max-Heapify` takes time $O(lg n)$.

## Topic: 11   Hash Tables

Many applications require a dynamic set that supports only the dictionary operations `Insert`, `Search`, and `Delete`. A hash table is an effective data structure for implementing dictionaries. Although searching for an element in a hash table can take as long as searching for an element in a linked list–$\Theta(n)$ time in the worst case—in practice, hashing performs extremely well. Under reasonable assumptions, the average time to search for an element in a hash table is $O(1)$.

A hash table generalizes the simpler notion of an ordinary array. Directly addressing into an ordinary array makes effective use of our ability to examine an arbitrary position in an array in $O(1)$ time. We can take advantage of direct addressing when we can afford to allocate an array that has one position for every possible key.

When the number of keys actually stored is small relative to the total number of possible keys, hash tables become an effective alternative to directly addressing an array, since a hash table typically uses an array of size proportional to the number of keys actually stored. Instead of using the key as an array index directly, the array index is computed from the key.

### Direct-adress tables

Direct addressing is a simple technique that works well when the universe $U$ of keys is reasonably small. Suppose that an application needs a dynamic set in which each element has a key drawn from the universe $U = \{0,1,...,m-1\}$, where $m$ is not too large. We shall assume that no two elements have the same key.

<img src="imgs/CFig11_1.png" alt="drawing" width="500"/>

In [9]:
class Key:
    def __init__(self, key, dato):
        self.key = key
        self.dato = dato
        
class DATable:
    def __init__(self, tam):
        self.T = [None] * Tam
    
    def Insert(self, x):
        self.T[x.key] = x
        
    def Delete(self, x):
        self.T[x.key] = None
    
    def Search(self, k):
        return self.T[k]

### Hash tables

The downside of direct addressing is obvious: if the universe $U$ is large, storing a table $T$ of size $|U|$ may be impractical, or even impossible, given the memory available on a typical computer. Furthermore, the set $K$ of keys actually stored may be so small relative to $U$ that most of the space allocated for $T$ would be wasted.

When the set $K$ of keys stored in a dictionary is much smaller than the universe $U$ of all possible keys, a hash table requires much less storage than a direct address table. Specifically, we can reduce the storage requirement to $\Theta(|K|)$ while we maintain the benefit that searching for an element in the hash table still requires only $O(1)$ time.

With direct addressing, an element with key $k$ is stored in slot $k$. With hashing, this element is stored in slot $h(k)$; that is, we use a **hash function** $h$ to compute the slot from the key $k$. Here, $h$ maps the universe $U$ of keys into the slots of a hash table $T[0..m-1]$.

Where the size $m$ of the hash table is typically much less than $|U|$. We say that an element with key $k$ hashes to slot $h(k)$; we also say that $h(k)$ is the hash value of key k.

<img src="imgs/CFig11_2.png" alt="drawing" width="500"/>

There is one hitch: two keys may hash to the same slot. We call this situation a collision. Fortunately, we have effective techniques for resolving the conflict created by collisions. In chaining, we place all the elements that hash to the same slot into the same linked list, as Figure 11.3 shows.
<img src="imgs/CFig11_3.png" alt="drawing" width="500"/>

### Hash functions

A good hash function satisfies (approximately) the assumption of simple uniform hashing: each key is equally likely to hash to any of the $m$ slots, independently of where any other key has hashed to. Unfortunately, we typically have no way to check this condition, since we rarely know the probability distribution from which the keys are drawn. Moreover, the keys might not be drawn independently.

#### The multiplication method
The multiplication method for creating hash functions operates in two steps. First, we multiply the key $k$ by a constant $A$ in the range $0 < A < 1$ and extract the fractional part of $kA$. Then, we multiply this value by $m$ and take the floor of the result. In short, the hash function is
$$h(k) = floor(m(kA mod 1))$$
Although this method works with any value of the constant A, it works better with some values than with others. The optimal choice depends on the characteristics of the data being hashed. Knuth  suggests that
$$A = (\sqrt{5} - 1)/2 = 0.6180339887..$$

In [10]:
m = 10

def h(k):
    A =   (math.sqrt(5) - 1) / 2
    return math.floor( m * ( (k*A) % 1 ) )

Podemos probar como va generando los hashes

In [11]:
for i in range(1, 11):
    print(h(i))

6
2
8
4
0
7
3
9
5
1


In [12]:
class HashTable:
    def __init__(self, m):
        self.T = [None] * m
        self.m = m
        
    # Hash prima
    def hP(self, k):
        A = (math.sqrt(5) - 1) / 2
        return math.floor( self.m * ( (k.key*A) % 1 ) )
    
    # Hash con linear probing
    def h(self, k, i):
        return (self.hP(k) + i) % self.m
    
    def Insert(self, k):
        i = 0
        while True:
            j = self.h(k, i)
            if self.T[j] == None:
                self.T[j] = k
                return j
            else:
                i = i + 1
            if(i == self.m):
                break
        print("error, hash table overflow")
    
    def Search(self, k):
        i = 0
        while True:
            j = self.h(k, i)
            if self.T[j] == k:
                return j
            i = i + 1
            if self.T[j] == None or i == m:
                break
        return None
            

In [13]:
tabla = HashTable(10)

tabla.Insert(Key(2, 100))
print(tabla.T)

k = Key(3, 100)
tabla.Insert(k)
print(tabla.T)

i = tabla.Search(k)
print(i)

[None, None, <__main__.Key object at 0x10e03c438>, None, None, None, None, None, None, None]
[None, None, <__main__.Key object at 0x10e03c438>, None, None, None, None, None, <__main__.Key object at 0x10e03c9b0>, None]
8


## Topic: 23 Minimum Spanning Trees and The algorithms of Kruskal and Prim

Electronic circuit designs often need to make the pins of several components electrically equivalent by wiring them together. To interconnect a set of $n$ pins, we can use an arrangement of $n-1$ wires, each connecting two pins. Of all such arrangements, the one that uses the least amount of wire is usually the most desirable.
We can model this wiring problem with a connected, undirected graph $G = (V, E)$, where $V$ is the set of pins, $E$ is the set of possible interconnections between pairs of pins, and for each edge $(u, v) \in E$, we have a weight $w(u, v)$, specifying the cost (amount of wire needed) to connect $u$ and $v$. We then wish to find an acyclic subset $T \subseteq E$ that connects all of the vertices.

Since T is acyclic and connects all of the vertices, it must form a tree, which we call a spanning tree since it “spans” the graph $G$. We call the problem of determining the tree $T$ **the minimum-spanning-tree problem**. Figure 23.1 shows an example of a connected graph and a minimum spanning tree.

<img src="imgs/CFig23_1.png" alt="drawing" width="500"/>

### Kruskal’s algorithm

Kruskal’s algorithm finds a safe edge to add to the growing forest by finding, of all the edges that connect any two trees in the forest, an edge $(u, v)$ of least weight. Kruskal’s algorithm qualifies as a greedy algorithm because at each step it adds to the forest an edge of least possible weight.

It uses a disjoint-set data structure to maintain several disjoint sets of elements. Each set contains the vertices in one tree of the current forest. The operation $Find-Set(u)$ returns a representative element from the set that contains $u$. Thus, we can determine whether two vertices $u$ and $v$ belong to the same tree by testing whether $Find-Set(u)$ equals $Find-Set(v)$ To combine trees, Kruskal’s algorithm calls the $Union$ procedure.

<img src="imgs/CPseudo_Kruskal.png" alt="drawing" width="450"/>

The loop checks, for each edge $(u, v)$, whether the endpoints $u$ and $v$ belong to the same tree. If they do, then the edge $(u, v)$ cannot be added to the forest without creating a cycle, and the edge is discarded. Otherwise, the two vertices belong to different trees. In this case, line 7 adds the edge $(u, v)$ to A, and line 8 merges the vertices in the two trees.

<img src="imgs/CFig23_4_1.png" alt="drawing" width="450"/>
<img src="imgs/CFig23_4_2.png" alt="drawing" width="450"/>

The running time of Kruskal’s algorithm for a graph $G = (V, E)$ depends on how we implement the disjoint-set data structure. We assume that we use the disjoint-set-forest implementation with the union-by-rank and path-compression heuristics, since it is the asymptotically fastest implementation known.

We can restate the running time of Kruskal’s algorithm as 
$$O(E * log V)$$

In [14]:
# Implementacion usando lo siguiente:
# Union por rango
# Compresion de camino

class Grafo:
    # Implementacion unica con numeros como vertices
    def __init__(self, vertices):
        self.V = vertices
        self.grafo = []
    
    def AddArista(self, u, v, w):
        self.grafo.append([u, v, w])
        
    # Busqueda usando compresion de camino
    def Busqueda(self, padre, i):
        if padre[i] == i:
            return i
        return self.Busqueda(padre, padre[i])
    
    # Union por rango
    def Union(self, padre, rango, x, y):
        xraiz = self.Busqueda(padre, x)
        yraiz = self.Busqueda(padre, y)
        
        if rango[xraiz] < rango[yraiz]:
            padre[xraiz] = yraiz
        elif rango[xraiz] > rango[yraiz]:
            padre[yraiz] = xraiz
        else:
            padre[yraiz] = xraiz
            rango[xraiz] += 1
            
    def KruskalMST(self):
        A = []     # resultado del MST
        i = 0
        e = 0
        
        # Ordena el grafo por su peso
        self.grafo =  sorted(self.grafo, key=lambda item: item[2])
        padre = [] ; rango = []
        
        for nodo in range(self.V):
            padre.append(nodo)
            rango.append(0)
            
        while e < self.V - 1:
            u, v, w = self.grafo[i]
            i = i + 1
            x = self.Busqueda(padre, u)
            y = self.Busqueda(padre, v)
            
            if x != y:
                e = e + 1
                A.append([u, v, w])
                self.Union(padre, rango, x, y)
            
        return A

In [15]:
# Probando el codigo
# a-0, b-1, c-2, d-3, e-4, f-5, g-6, h-7, i-8 
g = Grafo(9)
g.AddArista(0, 1, 4)
g.AddArista(1, 2, 8)
g.AddArista(2, 3, 7)
g.AddArista(3, 4, 9)
g.AddArista(4, 5, 10)
g.AddArista(5, 6, 2)
g.AddArista(6, 7, 1)
g.AddArista(7, 0, 8)
g.AddArista(7, 8, 7)
g.AddArista(6, 8, 6)
g.AddArista(8, 2, 2)
g.AddArista(2, 5, 4)
g.AddArista(1, 7, 11)
g.AddArista(3, 5, 14)

A = g.KruskalMST()

print(A)

[[6, 7, 1], [5, 6, 2], [8, 2, 2], [0, 1, 4], [2, 5, 4], [2, 3, 7], [1, 2, 8], [3, 4, 9]]


### Prim’s algorithm

Prim’s algorithm operates much like Dijkstra’s algorithm for finding shortest paths in a graph. Prim’s algorithm has the property that the edges in the set A always form a single tree.
As Figure 23.5 shows, the tree starts from an arbitrary root vertex $r$ and grows until the tree spans all the vertices in $V$. Each step adds to the tree $A$ a light edge that connects $A$ to an isolated vertex—one on which no edge of $A$ is incident.
In order to implement Prim’s algorithm efficiently, we need a fast way to select a new edge to add to the tree formed by the edges in $A$. In the pseudocode below, the connected graph $G$ and the root $r$ of the minimum spanning tree to be grown are inputs to the algorithm. During execution of the algorithm, all vertices that are not in the tree reside in a min-priority queue $Q$ based on a key attribute. For each vertex $v$, the attribute  $v.key$ is the minimum weight of any edge connecting $v$ to a vertex in the tree; by convention, $v.key = \infty$ if there is no such edge. The attribute $v.\pi$ names the parent of $v$ in the tree.

<img src="imgs/CPseudo_Prim.png" alt="drawing" width="300"/>
<img src="imgs/CFig23_5.png" alt="drawing" width="450"/>


In [16]:
class Grafo:
    # Implementacion unica con numeros como vertices
    def __init__(self, vertices):
        self.V = vertices
        self.grafo = []
        self.adjMtrx = [[0 for column in range(vertices)]  for row in range(vertices)]
    
    def AddArista(self, u, v, w):
        self.grafo.append([u, v, w])
        
    def CreaMtrxAdj(self):   
      for i in range(0, len(self.grafo)):
        self.adjMtrx[self.grafo[i][0]][self.grafo[i][1]] = self.grafo[i][2]
        self.adjMtrx[self.grafo[i][1]][self.grafo[i][0]] = self.grafo[i][2]
        
    def MinKey(self, key, mstSet):
        min = math.inf
        for v in range(self.V):
            if key[v] < min and mstSet[v] == False:
                min = key[v]
                min_index = v
        return min_index
        
    def PrimMST(self):
        key = [math.inf] * self.V
        parent = [None] * self.V
        
        key[0] = 0
        mstSet = [False] * self.V
        
        parent[0] = -1
        
        for cout in range(self.V): 
            u = self.MinKey(key, mstSet) 
            mstSet[u] = True
  
            for v in range(self.V): 
                if self.adjMtrx[u][v] > 0 and mstSet[v] == False and key[v] > self.adjMtrx[u][v]: 
                        key[v] = self.adjMtrx[u][v] 
                        parent[v] = u 
                        
        A = []
        for i in range(1, self.V): 
            A.append([parent[i], i, self.adjMtrx[i][parent[i]]])
        
        return A

In [17]:
g = Grafo(9)

g.AddArista(0, 1, 4)
g.AddArista(1, 2, 8)
g.AddArista(2, 3, 7)
g.AddArista(3, 4, 9)
g.AddArista(4, 5, 10)
g.AddArista(5, 6, 2)
g.AddArista(6, 7, 1)
g.AddArista(7, 0, 8)
g.AddArista(7, 8, 7)
g.AddArista(6, 8, 6)
g.AddArista(8, 2, 2)
g.AddArista(2, 5, 4)
g.AddArista(1, 7, 11)
g.AddArista(3, 5, 14)

g.CreaMtrxAdj()
A = g.PrimMST()
print(A)

[[0, 1, 4], [1, 2, 8], [2, 3, 7], [3, 4, 9], [2, 5, 4], [5, 6, 2], [6, 7, 1], [2, 8, 2]]
